# ETL DATA

IDEA: 
- Get 1000 artists, albums, tracks from year 2023
- Enrich artists from albums and tracks
- Get albums from artists
- Get tracks from albums
- Get audio features

Hwever, for simplicity (since the API has limit), will only implement the first point to run periodically. The rest will run manually.

In [ ]:
import pandas as pd
artist.to_csv('artist_df.csv', index=False)
album.to_csv('album_df.csv', index=False)
audio.to_csv('audio_df.csv', index=False)
track.to_csv('track_df.csv', index=False)

In [ ]:
import requests

# secret for spotify
cid1 = '' # replace with your own credential
secret1 = '' # replace with your own credential
cid2 = '' # replace with your own credential
secret2 = '' # replace with your own credential
cid3 = '' # replace with your own credential
secret3 = '' # replace with your own credential
cid4 = '' # replace with your own credential
secret4 = '' # replace with your own credential
cid5 = '' # replace with your own credential
secret5 = '' # replace with your own credential


cids = [cid1, cid2, cid3, cid4, cid5]
secrets = [secret1, secret2, secret3, secret4, secret5]

AUTH_URL = 'https://accounts.spotify.com/api/token'
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

def get_headers(CLIENT_ID, CLIENT_SECRET):
    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()

    # save the access token
    access_token = auth_response_data['access_token']
    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)
    }

    return headers

In [ ]:
from datetime import datetime
import pandas as pd
from tqdm import tqdm
import requests

### Get 1000 artists, albums, tracks from year 2023



In [ ]:
album_id = []
track_name = []
popularity = []
track_id = []
years = ['2021','2022','2023']
artist_id = []
headers = get_headers(cids[0], secrets[0])
# Operations
for year in tqdm(years):
    query = f'year:{year}'
    for i in range(0,1000,50):
        params = {'q': query, 'type': 'track', 'limit': 50, 'offset':i}
        track_results = requests.get(f"{BASE_URL}search/", headers=headers, params=params).json()
        print(track_results)
        for i, t in enumerate(track_results['tracks']['items']):
            if t['id'] in track_id:
                continue
            artist_id.append(t['artists'][0]['id'])
            album_id.append(t['album']['id'])
            track_name.append(t['name'])
            track_id.append(t['id'])
            popularity.append(t['popularity'])

track_dataframe = pd.DataFrame({'track_id' : track_id, 'artist_id': artist_id, 'track_name' : track_name, 'popularity' : popularity, 'album_id': album_id})

In [ ]:
track_dataframe

,track_id,artist_id,track_name,popularity,album_id
0,6AQbmUe0Qwf5PZnt4HmTXv,78rUTD7y6Cy67W1RVzYs7t,Boy's a liar Pt. 2,96,6cVfHBcp3AdpYY0bBglkLN
1,0yLdNVWF3Srea0uzk55zFn,5YGY8feqx7naU7z4HrwZM6,Flowers,100,7I0tjwFtxUwBC1vgyeMAax
2,59uQI0PADDKeE6UZDTJEe8,4oUHIQIBe0LHzYfvXNW4QM,Last Night,88,7fOmdhRrRohTzToL617xkk
3,7oDd86yk8itslrA9HRP2ki,1Xyo4u8uXC1ZmMpatF05PJ,Die For You - Remix,95,6Exo0MYoL3XammoTDeihFy
4,7K3BhSpAxZBznislvUMVtn,4oUHIQIBe0LHzYfvXNW4QM,Last Night,85,6i7mF7whyRJuLJ4ogbH2wh
...,...,...,...,...,...
995,1qPWymsBDrRXApPGdxI1E6,0C8ZW7ezQVs4URX5aX7Kqx,Lose You To Love Me,0,0CBha5lrgwky50LASMRWL1
996,3zuujAYpvDLwLmO2MpavXT,0C8ZW7ezQVs4URX5aX7Kqx,Slow Down - DJLW Remix,0,1sFqsST4ozmIeM2ed3Vaon
997,0D8uUFo8NGmH396z0WVmZq,0C8ZW7ezQVs4URX5aX7Kqx,Back To You - Joey Pecoraro Remix,0,6pI2tEV4SFqOIpZf8Z3PSf
998,2QgQXG9o2kNencaRSOdOEN,0C8ZW7ezQVs4URX5aX7Kqx,Wolves,0,3SzElkgbcuXILSeT5Ea6kC


In [ ]:
artist_name = []
artist_id = []
popularity = []
genre = []
years = ['2023']
headers = get_headers(cids[0], secrets[0])
# Operations
for year in tqdm(years):
    query = f'year:{year}'
    for i in range(0,1000,50):
        try:
            params = {'q': query, 'type': 'artist', 'limit': 50, 'offset':i}
            results = requests.get(f"{BASE_URL}search/", headers=headers, params=params).json()
        except:
            continue
        if 'artists' not in results:
            continue
        for i, t in enumerate(results['artists']['items']):
            if t['id'] in artist_id:
                continue
            artist_name.append(t['name'])
            artist_id.append(t['id'])
            popularity.append(t['popularity'])
            genre.append(t['genres'])

artist_df = pd.DataFrame({'artist_id' : artist_id, 'artist_name' : artist_name, 'genre': genre, 'popularity' : popularity})
artist_df_json = artist_df.to_json(orient='split')

100%|██████████| 1/1 [00:14<00:00, 14.22s/it]


In [ ]:
artist_df

,artist_id,artist_name,genre,popularity
0,4oUHIQIBe0LHzYfvXNW4QM,Morgan Wallen,[contemporary country],94
1,3TVXtAsR1Inumwj472S9r4,Drake,"[canadian hip hop, canadian pop, hip hop, rap,...",98
2,06HL4z0CvFAxyc27GXpf02,Taylor Swift,[pop],100
3,1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,"[canadian contemporary r&b, canadian pop, pop]",99
4,7tYKF4w9nC0nq9CsPZTHyP,SZA,"[pop, r&b]",94
...,...,...,...,...
795,7pwufEBGfggjoI8twqlsmQ,Nessa Barrett,"[alt z, electropop, pop, social media pop]",71
796,3XHO7cRUPCLOr6jwp8vsx5,alt-J,"[indie rock, modern rock]",72
797,2TI7qyDE0QfyOlnbtfDo7L,Dave Matthews Band,"[jam band, neo mellow, pop rock]",66
798,1gJdf4Yybu4X5A2xYV3NMV,Eden Muñoz,"[nueva musica mexicana, urbano mexicano]",76


In [ ]:
artist_id = []
album_name = []
release_date = []
total_tracks = []
genre = []
popularity = []
album_id = []
headers = get_headers(cids[0], secrets[0])
# Operations
for i in range(0,1000,50):
    
    query = 'year:2023'
    params = {'q': query, 'type': 'album', 'limit': 50, 'offset':i}
    album_results = requests.get(f"{BASE_URL}search/", headers=headers, params=params).json()
    # album_results = sp.search(q='year:2023', type='album', market='SG', limit=50,offset=i)
    items = album_results['albums']['items']
    items = list(filter(lambda x:x, items))
    if items:
      album_id.extend(list(map(lambda x:x['id'], items)))
      artist_id.extend(list(map(lambda x:x['artists'][0]['id'], items)))
      album_name.extend(list(map(lambda x:x['name'], items)))
      release_date.extend(list(map(lambda x:x['release_date'], items)))
      total_tracks.extend(list(map(lambda x:x['total_tracks'], items)))

j = 0
for id in album_id:
    albumId = id
    url = f'https://api.spotify.com/v1/albums/{albumId}'

    features = False
    pop = 0
    # Make the GET request
    try:
        features = requests.get(url, headers=headers).json()
        pop = features['popularity']
    except Exception as e:
        print(e)

    popularity.append(pop)

# Create Dataframe
album_df = pd.DataFrame({'id': album_id, 'artist_id' : artist_id, 'album_name' : album_name, 'total_tracks' : total_tracks, 'release_date' : release_date, 'popularity': popularity})
print("album: ", album_df.shape)
print("error", j)
            

album:  (998, 6)
error 0


In [ ]:
album_df

,id,artist_id,album_name,total_tracks,release_date,popularity
0,6i7mF7whyRJuLJ4ogbH2wh,4oUHIQIBe0LHzYfvXNW4QM,One Thing At A Time,36,2023-03-03,98
1,4kS7bSuU0Jm9LYMosFU2x5,790FomKkXshlbRYZFtlgla,MAÑANA SERÁ BONITO,17,2023-02-24,98
2,0HiZ8fNXwJOQcrf5iflrdz,5YGY8feqx7naU7z4HrwZM6,Endless Summer Vacation,13,2023-03-10,92
3,6cVfHBcp3AdpYY0bBglkLN,78rUTD7y6Cy67W1RVzYs7t,Boy's a liar Pt. 2,2,2023-02-03,90
4,25Uddgldy3slnChqKqHsIM,3qiHUAX7zY4Qnjx8TNUzVx,AftërLyfe,22,2023-02-24,86
...,...,...,...,...,...,...
993,4fjkwwqP8EzhuU8SP9P31j,7DkseLyOZrdRjCuoWFtqFi,Serás Mía (En Vivo),1,2023-03-24,0
994,6Xnrf8JvckhoADFcePUNyq,4Gso3d4CscCijv0lmajZWs,Love Sick,16,2023-02-23,58
995,4vIEdzVa15kQvoalvxPIz5,09O5HSw8RO3aAnWNCADaKS,BUBBLY,1,2023-02-28,59
996,50CjJwLpGUfQocgHP80Xve,4bzfsZhaLW6VWHLh1sqcrK,Past // Present // Future,11,2023-03-10,55


### Enrich artists from albums and tracks

In [ ]:
def enrich_artist_from_other_df(artist_df, other_df):
    headers = get_headers(cids[1], secrets[1])
    artist_name = []
    artist_id = []
    popularity = []
    genre = []
    curr = list(artist_df['artist_id'])
    oth = list(set(other_df['artist_id']))
    count = 0
    for y in tqdm(range(0,len(oth),50)):
        x = oth[y:min(y+50, len(oth))]
        x = ','.join(x)
        params = f"ids={x}"
        if x in curr:
            continue
        result = requests.get(BASE_URL + 'artists?'+params, headers=headers)
        result = result.json()
        if 'artists' not in result:
          continue
        else:
          result = result['artists']
        artist_name.extend(list(map(lambda x:x['name'], result)))
        artist_id.extend(list(map(lambda x:x['id'], result)))
        popularity.extend(list(map(lambda x:x['popularity'], result)))
        genre.extend(list(map(lambda x:x['genres'], result)))
        count += 1
    
    artist_dataframe = pd.DataFrame({'artist_id' : artist_id, 'artist_name' : artist_name, 'genre': genre, 'popularity' : popularity})
    artist_df = artist_df.append(artist_dataframe)
    return artist_df

In [ ]:
artist_df = enrich_artist_from_other_df(artist_df, track_dataframe)

100%|██████████| 3/3 [00:00<00:00,  9.88it/s]
<ipython-input-9-4abe4458bda0>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  artist_df = artist_df.append(artist_dataframe)


In [ ]:
artist_df

,artist_id,artist_name,genre,popularity
0,4oUHIQIBe0LHzYfvXNW4QM,Morgan Wallen,[contemporary country],94
1,3TVXtAsR1Inumwj472S9r4,Drake,"[canadian hip hop, canadian pop, hip hop, rap,...",98
2,06HL4z0CvFAxyc27GXpf02,Taylor Swift,[pop],100
3,1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,"[canadian contemporary r&b, canadian pop, pop]",99
4,7tYKF4w9nC0nq9CsPZTHyP,SZA,"[pop, r&b]",94
...,...,...,...,...
140,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,"[dance pop, pop]",90
141,3tJoFztHeIJkJWMrx0td2f,Moneybagg Yo,"[hip hop, memphis hip hop, rap, southern hip h...",78
142,7kNqXtgeIwFtelmRjWv205,John Summit,"[edm, electro house, house, pop dance]",69
143,4xRYI6VqpkE3UwrDrAZL8L,Logic,"[conscious hip hop, dmv rap, hip hop, pop rap,...",80


In [ ]:
artist_df = enrich_artist_from_other_df(artist_df, album_df)

100%|██████████| 14/14 [00:01<00:00,  9.64it/s]
<ipython-input-9-4abe4458bda0>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  artist_df = artist_df.append(artist_dataframe)


In [ ]:
artist_df

,artist_id,artist_name,genre,popularity
0,4oUHIQIBe0LHzYfvXNW4QM,Morgan Wallen,[contemporary country],94
1,3TVXtAsR1Inumwj472S9r4,Drake,"[canadian hip hop, canadian pop, hip hop, rap,...",98
2,06HL4z0CvFAxyc27GXpf02,Taylor Swift,[pop],100
3,1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,"[canadian contemporary r&b, canadian pop, pop]",99
4,7tYKF4w9nC0nq9CsPZTHyP,SZA,"[pop, r&b]",94
...,...,...,...,...
668,0dXXFVCw0LKzmHFrTLUaQJ,Tito Torbellino Jr,"[corrido, musica mexicana, nueva musica mexicana]",66
669,77IKXFvO7SpWrq8hflrUXc,KB,"[christian hip hop, christian trap]",63
670,5yxyJsFanEAuwSM5kOuZKc,London Symphony Orchestra,"[british orchestra, classical performance, orc...",75
671,5VWWftIdcTzXDWTicX5l0c,Adumbration,[],48


### Get albums from artists

In [ ]:
album_name = []
release_date = []
total_tracks = []
genre = []
popularity = []
album_id = []
art_id = []
headers = get_headers(cids[2], secrets[2])
for x in tqdm(range(artist_df.shape[0])):
    artist_id = artist_df.iloc[x]['artist_id']
    albums = requests.get(f"{BASE_URL}artists/{artist_id}/albums", headers=headers).json()
    if 'items' not in albums:
      continue
    albums = albums['items']
    items = list(filter(lambda x:x, albums))[:3]
    if items:
      album_id.extend(list(map(lambda x:x['id'], items)))
      art_id.extend([artist_id for x in range(len(items))])
      album_name.extend(list(map(lambda x:x['name'], items)))
      release_date.extend(list(map(lambda x:x['release_date'], items)))
      total_tracks.extend(list(map(lambda x:x['total_tracks'], items)))

100%|██████████| 1618/1618 [04:59<00:00,  5.41it/s]


In [ ]:
j = 0
for id in tqdm(album_id[:len(album_id)//2]):
    albumId = id
    url = f'https://api.spotify.com/v1/albums/{albumId}'

    features = False
    pop = 0
    # Make the GET request
    try:
        features = requests.get(url, headers=headers).json()
        pop = features['popularity']
    except Exception as e:
        print(e)

    popularity.append(pop)

100%|██████████| 2409/2409 [05:21<00:00,  7.50it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import joblib
output_dir = '/content/drive/MyDrive/IS3107'
joblib.dump(track_dataframe, 'track_dataframe.pkl')
joblib.dump(album_df, 'album_df.pkl')
joblib.dump(artist_df, 'artist_df.pkl')

['artist_df.pkl']

In [ ]:
j = 0
headers = get_headers(cids[3], secrets[3])
for id in tqdm(album_id[len(album_id)//2:]):
    albumId = id
    url = f'https://api.spotify.com/v1/albums/{albumId}'

    features = False
    pop = 0
    # Make the GET request
    try:
        features = requests.get(url, headers=headers).json()
        pop = features['popularity']
    except Exception as e:
        print(e)

    popularity.append(pop)

album_dataframe = pd.DataFrame({'id': album_id, 'artist_id' : art_id, 'album_name' : album_name, 'total_tracks' : total_tracks, 'release_date' : release_date, 'popularity': popularity})
album_df = album_df.append(album_dataframe)
album_df = album_df.drop_duplicates()

100%|██████████| 2409/2409 [05:02<00:00,  7.95it/s]
<ipython-input-18-80099226f5f6>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  album_df = album_df.append(album_dataframe)


In [ ]:
album_df

,id,artist_id,album_name,total_tracks,release_date,popularity
0,6i7mF7whyRJuLJ4ogbH2wh,4oUHIQIBe0LHzYfvXNW4QM,One Thing At A Time,36,2023-03-03,98
1,4kS7bSuU0Jm9LYMosFU2x5,790FomKkXshlbRYZFtlgla,MAÑANA SERÁ BONITO,17,2023-02-24,98
2,0HiZ8fNXwJOQcrf5iflrdz,5YGY8feqx7naU7z4HrwZM6,Endless Summer Vacation,13,2023-03-10,92
3,6cVfHBcp3AdpYY0bBglkLN,78rUTD7y6Cy67W1RVzYs7t,Boy's a liar Pt. 2,2,2023-02-03,90
4,25Uddgldy3slnChqKqHsIM,3qiHUAX7zY4Qnjx8TNUzVx,AftërLyfe,22,2023-02-24,86
...,...,...,...,...,...,...
4807,4F78fdhOqb861NcldgbPTH,77IKXFvO7SpWrq8hflrUXc,Today We Rebel,14,2017-10-20,49
4808,5mMhmWAzd3vTvhzShSoiXx,77IKXFvO7SpWrq8hflrUXc,Today We Rebel (Commentary),13,2017-10-19,16
4813,1lFzJaaCrSAoFfqvAQWu0e,5VWWftIdcTzXDWTicX5l0c,Connections,1,2020-10-01,15
4814,6Vz2CBGS1UcQ1GDTopeXbF,5VWWftIdcTzXDWTicX5l0c,Serenitivity,1,2020-06-03,17


In [ ]:
joblib.dump(album_df, 'album_df.pkl')

['album_df.pkl']

### Get tracks from albums

In [ ]:
track_name = []
popularity = []
track_id = []
artist_id = []
album_id = []
headers = get_headers(cids[4], secrets[4])
for x in tqdm(range(album_df.shape[0])):
    art_id = album_df.iloc[x]['artist_id']
    alb_id = album_df.iloc[x]['id']
    tracks = requests.get(f"{BASE_URL}albums/{alb_id}/tracks", headers=headers).json()
    if 'items' not in tracks:
      continue
    items = list(filter(lambda x:x, tracks['items']))[:2]
    if items:
      track_id.extend(list(map(lambda x:x['id'], items)))
      artist_id.extend([art_id for x in range(len(items))])
      album_id.extend([alb_id for x in range(len(items))])
      track_name.extend(list(map(lambda x:x['name'], items)))


100%|██████████| 4048/4048 [07:59<00:00,  8.43it/s]


In [ ]:
for y in tqdm(range(0,len(track_id),50)):
    x = track_id[y:min(y+50, len(track_id))]
    x = ','.join(x)
    params = f"ids={x}"
    result = requests.get(BASE_URL + 'tracks?'+params, headers=headers)
    result = result.json()
    for i, t in enumerate(result['tracks']):
      popularity.append(t['popularity'])

track_df = pd.DataFrame({'track_id' : track_id, 'artist_id': artist_id, 'track_name' : track_name, 'popularity' : popularity, 'album_id': album_id})
track_dataframe = track_dataframe.append(track_df)
track_dataframe.drop_duplicates()

100%|██████████| 147/147 [00:34<00:00,  4.27it/s]


In [ ]:
track_df = pd.DataFrame({'track_id' : track_id, 'artist_id': artist_id, 'track_name' : track_name, 'popularity' : popularity, 'album_id': album_id})
track_dataframe = track_dataframe.append(track_df)
track_dataframe.drop_duplicates()

<ipython-input-30-4c24008f71aa>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  track_dataframe = track_dataframe.append(track_df)


,track_id,artist_id,track_name,popularity,album_id
0,6AQbmUe0Qwf5PZnt4HmTXv,78rUTD7y6Cy67W1RVzYs7t,Boy's a liar Pt. 2,96,6cVfHBcp3AdpYY0bBglkLN
1,0yLdNVWF3Srea0uzk55zFn,5YGY8feqx7naU7z4HrwZM6,Flowers,100,7I0tjwFtxUwBC1vgyeMAax
2,59uQI0PADDKeE6UZDTJEe8,4oUHIQIBe0LHzYfvXNW4QM,Last Night,88,7fOmdhRrRohTzToL617xkk
3,7oDd86yk8itslrA9HRP2ki,1Xyo4u8uXC1ZmMpatF05PJ,Die For You - Remix,95,6Exo0MYoL3XammoTDeihFy
4,7K3BhSpAxZBznislvUMVtn,4oUHIQIBe0LHzYfvXNW4QM,Last Night,85,6i7mF7whyRJuLJ4ogbH2wh
...,...,...,...,...,...
7325,78NvEoQpN4pOGLWrmrw9AM,77IKXFvO7SpWrq8hflrUXc,Tempo (Commentary),6,5mMhmWAzd3vTvhzShSoiXx
7326,70UILYcP8Q8kvQhFKvLP5o,5VWWftIdcTzXDWTicX5l0c,Connections,21,1lFzJaaCrSAoFfqvAQWu0e
7327,41hD0lo1wIP9GX25JE4Kfu,5VWWftIdcTzXDWTicX5l0c,Serenitivity,23,6Vz2CBGS1UcQ1GDTopeXbF
7328,04WLCoFu2UZg8u2dwNE71v,6UnhGhByBdAIpd9ZEtYO1g,ZZAFFRIC RHYTHM$lOWER,48,6taaDXI7sNhT7NMeSI6lvj


In [ ]:
joblib.dump(track_dataframe, 'track_df.pkl')

['track_df.pkl']

In [ ]:
track_dataframe = track

### Get audio features

In [ ]:
audio_features = []
headers = get_headers(cids[1], secrets[1])
track_id = track_dataframe['track_id']
# Operations
for y in range(0,len(track_id),50):
    x = track_id[y:min(y+50, len(track_id))]
    x = ','.join(x)
    params = f"ids={x}"
    url = f'{BASE_URL}audio-features?{params}'

    # Make the GET request
    features = requests.get(url, headers=headers).json()
    if 'audio_features' not in features:
        continue
    # features = sp.audio_features(id)
    if features['audio_features']:
        audio_features.extend(features['audio_features'])

In [ ]:
audio_dataframe = pd.DataFrame(list(filter(lambda x:x, audio_features)))
print("audio: ", audio_dataframe.shape)

audio:  (7526, 18)


In [ ]:
audio_dataframe

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.696,0.809000,5,-8.254,1,0.0500,0.2520,0.000128,0.2480,0.8570,132.962,audio_features,6AQbmUe0Qwf5PZnt4HmTXv,spotify:track:6AQbmUe0Qwf5PZnt4HmTXv,https://api.spotify.com/v1/tracks/6AQbmUe0Qwf5...,https://api.spotify.com/v1/audio-analysis/6AQb...,131013,4
1,0.707,0.681000,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.6460,117.999,audio_features,0yLdNVWF3Srea0uzk55zFn,spotify:track:0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://api.spotify.com/v1/audio-analysis/0yLd...,200455,4
2,0.517,0.675000,6,-5.382,1,0.0357,0.4590,0.000000,0.1510,0.5180,203.853,audio_features,59uQI0PADDKeE6UZDTJEe8,spotify:track:59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://api.spotify.com/v1/audio-analysis/59uQ...,163855,4
3,0.531,0.525000,1,-6.500,0,0.0671,0.2320,0.000000,0.4410,0.5020,66.900,audio_features,7oDd86yk8itslrA9HRP2ki,spotify:track:7oDd86yk8itslrA9HRP2ki,https://api.spotify.com/v1/tracks/7oDd86yk8its...,https://api.spotify.com/v1/audio-analysis/7oDd...,232857,4
4,0.492,0.675000,6,-5.456,1,0.0389,0.4670,0.000000,0.1420,0.4780,203.759,audio_features,7K3BhSpAxZBznislvUMVtn,spotify:track:7K3BhSpAxZBznislvUMVtn,https://api.spotify.com/v1/tracks/7K3BhSpAxZBz...,https://api.spotify.com/v1/audio-analysis/7K3B...,163855,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7521,0.799,0.361000,1,-11.569,1,0.3970,0.1450,0.000000,0.1080,0.3300,111.104,audio_features,78NvEoQpN4pOGLWrmrw9AM,spotify:track:78NvEoQpN4pOGLWrmrw9AM,https://api.spotify.com/v1/tracks/78NvEoQpN4pO...,https://api.spotify.com/v1/audio-analysis/78Nv...,161838,5
7522,0.198,0.000956,9,-36.318,0,0.0486,0.9900,0.905000,0.1010,0.1110,106.713,audio_features,70UILYcP8Q8kvQhFKvLP5o,spotify:track:70UILYcP8Q8kvQhFKvLP5o,https://api.spotify.com/v1/tracks/70UILYcP8Q8k...,https://api.spotify.com/v1/audio-analysis/70UI...,169673,3
7523,0.198,0.065700,7,-20.771,1,0.0375,0.9830,0.963000,0.0684,0.0372,64.911,audio_features,41hD0lo1wIP9GX25JE4Kfu,spotify:track:41hD0lo1wIP9GX25JE4Kfu,https://api.spotify.com/v1/tracks/41hD0lo1wIP9...,https://api.spotify.com/v1/audio-analysis/41hD...,213788,4
7524,0.743,0.223000,2,-14.090,0,0.1520,0.0861,0.848000,0.0918,0.8890,187.942,audio_features,04WLCoFu2UZg8u2dwNE71v,spotify:track:04WLCoFu2UZg8u2dwNE71v,https://api.spotify.com/v1/tracks/04WLCoFu2UZg...,https://api.spotify.com/v1/audio-analysis/04WL...,30672,4


In [ ]:
joblib.dump(audio_dataframe, 'audio_df.pkl')

['audio_df.pkl']

In [ ]:
artist_df = artist_df.drop_duplicates(subset=['artist_id']).reset_index()
artist_df = artist_df[artist_df['popularity']!=0]
album_df = album_df[album_df['popularity']!=0].reset_index()
track_dataframe = track_dataframe[track_dataframe['popularity']!=0].reset_index()

In [ ]:
joblib.dump(audio_dataframe, 'audio_df.pkl')
joblib.dump(track_dataframe, 'track_df.pkl')
joblib.dump(artist_df, 'artist_df.pkl')
joblib.dump(album_df, 'album_df.pkl')

['album_df.pkl']